### Installing modules and dependencies


In [ ]:
# Installing modules
%pip install torch torchvision torchaudio
%pip install pyqt5 lxml --upgrade

In [ ]:
# Cloning the YOLOv5 repository to our device
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# Installing other requirements
!cd yolov5
%pip install -r yolov5\requirements.txt

### Loading the model and making sure YOLO works on webcam

In [8]:
# Imports
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import uuid # Unique identifier
import os
import time

In [ ]:
# Defining our model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
print(model)

In [ ]:
# Checking whether YOLOv5 is working or not

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### Preparing data (can also be done using script "capture-data.py")

In [63]:
# Defining data directory and classes for prediction

IMAGES_PATH = os.path.join('data', 'images') # /data/images
labels = ['Covering mouth (WARNING!)', 'speaking or laughing (WARNING!)' , 'looking on screen or thinking (Normal)']
number_imgs = 15

In [64]:
# Capturing images for fine-tuning YOLOv5 model

cap = cv2.VideoCapture(0)
# Loop through labels
for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(4)
    
    # Loop through image range
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        
        # Webcam feed
        ret, frame = cap.read()
        
        # Naming out image path
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        
        # Writes out image to file 
        cv2.imwrite(imgname, frame)
        
        # Render to the screen
        cv2.imshow('Image Collection', frame)
        
        # 2 second delay between captures
        time.sleep(2)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [101]:
# Concatenating image name and path
print(os.path.join(IMAGES_PATH, labels[0]+'.'+str(uuid.uuid1())+'.jpg'))

data\images\Covering mouth (WARNING!).07885216-87c6-11ef-9250-b48c9d378590.jpg


In [65]:
for label in labels:
    print('Collecting images for {}'.format(label))
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        print(imgname)   

data\images\Covering mouth (WARNING!).bfda038a-87ff-11ef-828d-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda038b-87ff-11ef-b237-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda038c-87ff-11ef-b6e3-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda038d-87ff-11ef-a8d8-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda038e-87ff-11ef-965d-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda038f-87ff-11ef-bf5b-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda0390-87ff-11ef-8e83-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda0391-87ff-11ef-a8ee-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda0392-87ff-11ef-8a60-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda0393-87ff-11ef-903d-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda0394-87ff-11ef-a45f-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda0395-87ff-11ef-b1f9-b48c9d378590.jpg
data\images\Covering mouth (WARNING!).bfda0396-87ff-

### Labelling Images

In [17]:
# Cloning Image Labelling GitHub repository
!git clone https://github.com/tzutalin/labelImg

Cloning into 'labelImg'...


In [1]:
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
!cd yolov5 && python train.py --img 320 --batch 16 --epochs 300 --data dataset.yml --weights yolov5s.pt --workers 2

### Loading and testing new YOLOv5 model in real-time

In [ ]:
new_model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp6/weights/last.pt', force_reload=True)

In [ ]:
# Testing the new YOLOv5 model

final_cap = cv2.VideoCapture(0)
final_cap.set(cv2.CAP_PROP_FPS, 30)

while final_cap.isOpened():
    ret2, frame2 = final_cap.read()
    
    # Make detections 
    final_results = new_model(frame2)
    
    cv2.imshow('VisionProctor', np.squeeze(final_results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
final_cap.release()
cv2.destroyAllWindows()

### Testing model on webcam-recorded video

In [ ]:
# Using model on webcam-recorded video

# Path to your input video file
video_path = 'demo-videos\input-video.mp4' 

# Start capturing from the video
final_cap = cv2.VideoCapture(video_path)

# Get video properties (width, height, FPS) for saving the output video
frame_width = int(final_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(final_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = final_cap.get(cv2.CAP_PROP_FPS)

# Create VideoWriter object to save output video
output_path = 'demo-videos\output_video.mp4'  # Path to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 video
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while final_cap.isOpened():
    ret2, frame2 = final_cap.read()
    
    if not ret2:
        break

    # Make detections
    final_results = new_model(frame2)
    
    # Get the frame with rendered bounding boxes
    result_frame = np.squeeze(final_results.render())
    
    # Write the frame to the output video file
    out.write(result_frame)
    
    # Display the resulting frame with bounding boxes (optional)
    cv2.imshow('VisionProctor', result_frame)
    
    # Exit on pressing 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Releasing resources
final_cap.release()
out.release()  # Releasing the video writer
cv2.destroyAllWindows()